In [1]:
import pandas as pd
#from lxml import etree
import requests
from bs4 import BeautifulSoup

In [2]:
import json
import unicodedata
from datetime import datetime

---- using HTML response ----

In [3]:
import locale
locale.getdefaultlocale()

('sv_SE', 'UTF-8')

In [4]:
#locale.setlocale(locale.LC_TIME, "sv_SE.UTF-8")
#locale.setlocale(locale.LC_ALL, 'sv_SE')
locale.setlocale(locale.LC_TIME, "sv_SE")
#locale.setlocale(locale.LC_ALL, '')

'sv_SE'

In [22]:
url = "https://www.hemnet.se/salda/bostader"
params= {
    #'housing_form_groups':'apartments',
    'location_ids':'17853',
    #'item_types':'bostadsratt',
    'by':'sale_date', #sort by sold date
    'order':'asc',
    'preferred_sorting':'true',
    'sold_age':'1d', #1d,4d,1w,1m and so on...
    'page':1
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
soup = BeautifulSoup(response.content, "html.parser")
response.url

'https://www.hemnet.se/salda/bostader?location_ids=17853&by=sale_date&order=asc&preferred_sorting=true&sold_age=1d&page=1'

In [23]:
#list to add our objects to
item_list=[]

def read_pagedata(soupref):
    #find the ul with the results list
    result_ul=soupref.find('ul','sold-results')

    #get the list items with our data (filter out the ads)
    result_li=result_ul('li', 'sold-results__normal-hit')

    #print(result_li)

    #start loop...
    for item in result_li:
        #a dict to collect all key/values in
        item_dict={}

        #get tracking data json for ids...
        tracking_data=item.attrs['data-tracking-data']
        tracking_data_json=json.loads(tracking_data)

        #add ids
        item_dict['listingId']=tracking_data_json['listingId']
        item_dict['saleId']=tracking_data_json['saleId']
        
        #get the anchor href for the url
        result_li_a=item.find('a','sold-property-link')
        item_dict['url']=result_li_a['href']

        #get our sale listing info
        result_price_info=result_li_a.find('div','sold-property-listing__location')

        #address
        address=result_li_a.find('h2','sold-property-listing__heading').string.strip()
        #address=address.replace(' / ','/') #unicode getting in the way again?
        item_dict['address']=address

        #fee
        fee_el=result_li_a.find('div','sold-property-listing__fee')
        if fee_el:
            fee=fee_el.string.strip()
            fee=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
            item_dict['fee']=fee

        #living area
        #area=result_li_a.find('div','sold-property-listing__area').string.strip()
        #area=int("".join(unicodedata.normalize("NFKD",fee).strip('kr/mån').split()))
        #item_dict['area']=area
        area_el=result_li_a.find('div','sold-property-listing__area')
        if area_el:
            area=area_el.text.strip()
            item_dict["unprocessed"]=area
            area_arr="".join(area.split('m²')).split()
            
            print(area_arr)

            span_el = result_li_a.find('span', 'listing-card__attribute--normal-weight')
            if span_el: 
                total_area=\
                    float(area_arr[0].replace(",", ".")) + \
                    float(area_arr[2].replace(",", "."))
                item_dict['living_area']=total_area
                if len(area_arr) > 4:
                    item_dict['rooms']=area_arr[3]
            else:
                item_dict['living_area']=area_arr[0]
                if len(area_arr) > 1:
                    item_dict['rooms']=area_arr[1] 
        
        #get our sale price info
        result_price_info=result_li_a.find('div','sold-property-listing__price-info')

        #ending date
        end_date_el=result_price_info.find('div',class_="sold-property-listing__sold-date")
        if end_date_el:
            end_date_str=end_date_el.string.strip()
            end_date_str=end_date_str.replace('Såld ','')
            end_date=datetime.strptime(end_date_str, '%d %B %Y')
            item_dict['end_date']=end_date

        #end price
        end_price_el=result_price_info.find('div',class_="sold-property-listing__subheading")
        if end_price_el:
            end_price_str=end_price_el.string.strip()
            end_price=int("".join(end_price_str.strip(' kr').replace('Slutpris ','').split()))
            item_dict['end_price']=end_price

        #end price/m2
        end_price_area_el=result_price_info.find('div',class_="sold-property-listing__price-per-m2")
        if end_price_area_el:
            end_price_area_str=end_price_area_el.string.strip()
            end_price_area=int("".join(unicodedata.normalize("NFKD",end_price_area_str).strip('kr/m2').split()))
            item_dict['end_price_area']=end_price_area
        
        #price change
        end_price_change_el=result_price_info.find('div',class_="sold-property-listing__price-change")
        if end_price_change_el:
            end_price_change_str=end_price_change_el.string.strip()
            end_price_change_str="".join(end_price_change_str.strip(' %').split())
            end_price_change=int(end_price_change_str[1:])
            end_price_change_sign=end_price_change_str[0]

            #calculate rate (percentage in float)
            if end_price_change !=0:
                item_dict['end_price_change_rate']=eval(f'100{end_price_change_sign}{end_price_change}')/100
            elif end_price_change==0:
                item_dict['end_price_change_rate']=100/100

            item_dict['end_price_change']=end_price_change
            item_dict['end_price_change_sign']=end_price_change_sign

        item_list.append(item_dict)

#print(item_list)

In [24]:
read_pagedata(soup)
next_link=soup.find('a','next_page')
while next_link:
    print(next_link['href'])
    next_response = requests.request("GET", "https://www.hemnet.se"+next_link['href'], headers=headers)
    next_soup = BeautifulSoup(next_response.content, "html.parser")
    read_pagedata(next_soup)
    next_link=next_soup.find('a','next_page')

['81', '3', 'rum']
['93,5', '4', 'rum']
['128', '2', 'rum']
['125', '+', '32', '6', 'rum']
['91', '3', 'rum']
['56', '2', 'rum']
['89', '+', '74', '4', 'rum']


In [25]:
df=pd.json_normalize(item_list)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   listingId              7 non-null      int64         
 1   saleId                 7 non-null      int64         
 2   url                    7 non-null      object        
 3   address                7 non-null      object        
 4   fee                    5 non-null      float64       
 5   unprocessed            7 non-null      object        
 6   living_area            7 non-null      object        
 7   rooms                  7 non-null      object        
 8   end_date               7 non-null      datetime64[ns]
 9   end_price              7 non-null      int64         
 10  end_price_area         5 non-null      float64       
 11  end_price_change_rate  7 non-null      float64       
 12  end_price_change       7 non-null      int64         
 13  end_price

In [26]:
df

,listingId,saleId,url,address,fee,unprocessed,living_area,rooms,end_date,end_price,end_price_area,end_price_change_rate,end_price_change,end_price_change_sign
0,19546515,7246768301309144827,https://www.hemnet.se/salda/lagenhet-3rum-henr...,Henriksdalsringen 93,4589.0,81 m²\n \n 3 rum,81,3,2023-01-24,3750000,46296.0,1.00,0,±
1,19341035,3637673289218611078,https://www.hemnet.se/salda/lagenhet-4rum-nack...,"Tollareslingan 26, 8tr",6013.0,"93,5 m²\n \n 4 rum","93,5",4,2023-01-24,5500000,58824.0,0.95,5,-
2,19549315,3299109706494617943,https://www.hemnet.se/salda/lagenhet-2rum-salt...,Sjökvarnsbacken 10,6966.0,128 m²\n \n 2 rum,128,2,2023-01-24,10350000,80859.0,1.00,0,±
3,18687625,1721365857750166344,https://www.hemnet.se/salda/villa-6rum-eknas-n...,Anarisvägen 8,NaN,125\n \n + 32 ...,157.0,6,2023-01-24,9000000,NaN,1.01,1,+
4,19450335,1084945785929677573,https://www.hemnet.se/salda/lagenhet-3rum-vast...,Ormingeringen 57B,6098.0,91 m²\n \n 3 rum,91,3,2023-01-24,2800000,30769.0,0.97,3,-
5,19469795,3211506034536138682,https://www.hemnet.se/salda/lagenhet-2rum-kvar...,Tre Kronors väg 17,3610.0,56 m²\n \n 2 rum,56,2,2023-01-24,3150000,56250.0,0.99,1,-
6,19448977,4202265191048780700,https://www.hemnet.se/salda/villa-4rum-nacka-o...,Örnvägen 5,NaN,89\n \n + 74 m...,163.0,4,2023-01-25,7250000,NaN,1.04,4,+


In [ ]:
df[df["fee"].isna()]

import json
import unicodedata
from datetime import datetime


#find the ul with the results list
result_ul=soup.find('ul','sold-results')

#get the list items with our data (filter out the ads)
result_li=result_ul('li', 'sold-results__normal-hit')

print(len(result_li))

#start loop...
for item in result_li:
    
    #get the anchor href for the url
    result_li_a=item.find('a','sold-property-link')

    #living area
    area=result_li_a.find('div','sold-property-listing__area').string.strip()
    area_arr="".join(area.split('m²')).split()
    rooms=area_arr[1]
    living_area=area_arr[0]
    #area=int("".join(unicodedata.normalize("NFKD",area).strip('kr/mån').split()))
    #address
    #address_str=result_price_info.find('div',class_='sold-property-listing__location').string.strip()
    #print(address_str)